In [1]:
#Internet Movie Database
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("IMDB-project").config("spark.some.config.option", "some-value").getOrCreate())
from pyspark.sql import StructType, StructField, StringType, IntegerType, FloatType, Window, Row, col, explode, split, desc, sum, row_number, when

In [2]:
# Зчитати файли
name_basics = spark.read.csv('./data/name.basics.tsv.gz', header= True, sep='\t')
title_ratings = spark.read.csv('./data/title.ratings.tsv.gz',  header= True, sep='\t')
title_principals = spark.read.csv('./data/title.principals.tsv.gz', header= True, sep='\t')
title_episode = spark.read.csv('./data/title.episode.tsv.gz', header= True, sep='\t')
title_crew = spark.read.csv('./data/title.crew.tsv.gz', header= True, sep='\t')
title_basics = spark.read.csv('./data/title.basics.tsv.gz', header= True, sep='\t')
title_akas = spark.read.csv('./data/title.akas.tsv.gz', header= True, sep='\t')

In [3]:
# Вивести перші 5 рядків з кожного файлу
print("name_basics:")
print(name_basics.show(5))

print("\ntitle_ratings:")
print(title_ratings.show(5))

print("\ntitle_principals:")
print(title_principals.show(5))

print("\ntitle_episode:")
print(title_episode.show(5))

print("\ntitle_crew:")
print(title_crew.show(5))

print("\ntitle_basics:")
print(title_basics.show(5))

print("\ntitle_akas:")
print(title_akas.show(5))

name_basics:
+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0057345,tt00544...|
|nm0000004|   John Belushi|     1949|     1982|actor,soundtrack,...|tt0078723,tt00779...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00839...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows

None

title_ratings:
+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7| 

Create appropriate schemas for datasets and corresponding DataFrames by reading data from the PC.

In [4]:
# Таблиця Name_Basics: Структура і створення ДатаФрейму
schema_name_basics = StructType([
    StructField("nconst", StringType(), True),
    StructField("primaryName", StringType(), True),
    StructField("birthYear", StringType(), True),
    StructField("deathYear", StringType(), True),
###    StructField("primaryProfession", StringType(), True),
    StructField("knownForTitles", StringType(), True)
])
name_basics_df = spark.read.csv("./data/name_basics.tsv", header=True, schema=schema_name_basics, sep='\t')
name_basics_df.show(10)

+---------+---------------+---------+---------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|      knownForTitles|
+---------+---------------+---------+---------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|
|nm0000004|   John Belushi|     1949|     1982|actor,soundtrack,...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|
|nm0000006| Ingrid Bergman|     1915|     1982|actress,soundtrac...|
|nm0000007|Humphrey Bogart|     1899|     1957|actor,soundtrack,...|
|nm0000008|  Marlon Brando|     1924|     2004|actor,soundtrack,...|
|nm0000009| Richard Burton|     1925|     1984|actor,soundtrack,...|
|nm0000010|   James Cagney|     1899|     1986|actor,soundtrack,...|
+---------+---------------+---------+---------+--------------------+


In [5]:
# Таблиця Title_Ratings: Структура і створення ДатаФрейму
schema_title_ratings = StructType([
    StructField("tconst", StringType(), True),
    StructField("averageRating", FloatType(), True),
    StructField("numVotes", IntegerType(), True)
])

# Asuming you have a DataFrame named df_title_ratings
title_ratings_df = spark.read.csv("./data/title_ratings.tsv", sep="\t", header=True, schema=schema_title_ratings)
title_ratings_df.show(10)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2007|
|tt0000002|          5.8|     270|
|tt0000003|          6.5|    1916|
|tt0000004|          5.5|     178|
|tt0000005|          6.2|    2696|
|tt0000006|          5.0|     183|
|tt0000007|          5.4|     843|
|tt0000008|          5.4|    2154|
|tt0000009|          5.3|     208|
|tt0000010|          6.9|    7359|
+---------+-------------+--------+


In [6]:
# Таблиця Title_Principals: Структура і створення ДатаФрейму
schema_title_principals = StructType([
    StructField("tconst", StringType(), True),
    StructField("ordering", IntegerType(), True),
    StructField("nconst", StringType(), True),
    StructField("category", StringType(), True),
###    StructField("job", StringType(), True),
###    StructField("characters", StringType(), True)
])

# Assuming you have a DataFrame named df_title_principals
title_principals_df = spark.read.csv("./data/title_principals.tsv", sep="\t", header=True, schema=schema_title_principals)
title_principals_df.show(10)

+---------+--------+---------+---------------+
|   tconst|ordering|   nconst|       category|
+---------+--------+---------+---------------+
|tt0000001|       1|nm1588970|           self|
|tt0000001|       2|nm0005690|       director|
|tt0000001|       3|nm0374658|cinematographer|
|tt0000002|       1|nm0721526|       director|
|tt0000002|       2|nm1335271|       composer|
|tt0000003|       1|nm0721526|       director|
|tt0000003|       2|nm1770680|       producer|
|tt0000003|       3|nm1335271|       composer|
|tt0000003|       4|nm5442200|         editor|
|tt0000004|       1|nm0721526|       director|
+---------+--------+---------+---------------+


In [7]:
# Таблиця Title_Episode: Структура і створення ДатаФрейму
###schema_title_episode = StructType([
###    StructField("tconst", StringType(), True),
###    StructField("parentTconst", StringType(), True),
###    StructField("seasonNumber", StringType(), True),
###    StructField("episodeNumber", StringType(), True)
###])

# Assuming you have a DataFrame named df_title_episode
###title_episode_df = spark.read.csv("./data/title_episode.tsv", sep="\t", header=True, schema=schema_title_episode)
###title_episode_df.show(10)

In [8]:
#Таблиця Title_Crew: Структура і створення ДатаФрейму
schema_title_crew = StructType([
    StructField("tconst", StringType(), True),
    StructField("directors", StringType(), True),
###    StructField("writers", StringType(), True)
])

# Assuming you have a DataFrame named df_title_crew
title_crew_df = spark.read.csv("./data/title_crew.tsv", sep="\t", header=True, schema=schema_title_crew)
title_crew_df.show(10)

+---------+-------------------+
|   tconst|          directors|
+---------+-------------------+
|tt0000001|          nm0005690|
|tt0000002|          nm0721526|
|tt0000003|          nm0721526|
|tt0000004|          nm0721526|
|tt0000005|          nm0005690|
|tt0000006|          nm0005690|
|tt0000007|nm0005690,nm0374658|
|tt0000008|          nm0005690|
|tt0000009|          nm0085156|
|tt0000010|          nm0525910|
+---------+-------------------+


In [9]:
#Таблиця Title_Basics: Структура і створення ДатаФрейму
schema_title_basics = StructType([
    StructField("tconst", StringType(), True),
    StructField("titleType", StringType(), True),
    StructField("primaryTitle", StringType(), True),
    StructField("originalTitle", StringType(), True),
    StructField("isAdult", IntegerType(), True),
    StructField("startYear", IntegerType(), True),
    StructField("endYear", StringType(), True),
    StructField("runtimeMinutes", IntegerType(), True),
    StructField("genres", StringType(), True)
])

# Assuming you have a DataFrame named df_title_basics
title_basics_df = spark.read.csv("./data/title_basics.tsv", sep="\t", header=True, schema=schema_title_basics)
title_basics_df.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [10]:
#Таблиця: Структура і створення ДатаФрейму
schema_title_akas = StructType([
    StructField("titleId", StringType(), True),
###    StructField("ordering", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("region", StringType(), True),
###    StructField("language", StringType(), True),
    StructField("types", StringType(), True),
###    StructField("attributes", StringType(), True),
###    StructField("isOriginalTitle", IntegerType(), True)
])

# Assuming you have a DataFrame named df_title_akas
title_akas_df = spark.read.csv("./data/title_akas.tsv", sep="\t", header=True, schema=schema_title_akas)
title_akas_df.show(10)

+---------+-----+--------------------+-----+
|  titleId|title|              region|types|
+---------+-----+--------------------+-----+
|tt0000001|    1|          Карменсіта|   UA|
|tt0000001|    2|          Carmencita|   DE|
|tt0000001|    3|Carmencita - span...|   HU|
|tt0000001|    4|          Καρμενσίτα|   GR|
|tt0000001|    5|          Карменсита|   RU|
|tt0000001|    6|          Carmencita|   US|
|tt0000001|    7|          Carmencita|   \N|
|tt0000001|    8|      カルメンチータ|   JP|
|tt0000002|    1|Le clown et ses c...|   \N|
|tt0000002|    2|Le clown et ses c...|   FR|
+---------+-----+--------------------+-----+


Get the general information about dataset. (Such as Schema, Columns, number of columns and rows etc.)

In [11]:
#Name_Basics:
print("Schema for Name_Basics:")
name_basics_df.printSchema()

print("\nColumns:")
print(name_basics_df.columns)

num_columns = len(name_basics_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = name_basics_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
name_basics_df.describe().show()

Schema for Name_Basics:
root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- knownForTitles: string (nullable = true)


Columns:
['nconst', 'primaryName', 'birthYear', 'deathYear', 'knownForTitles']

Number of columns: 5

Number of rows: 13040654

Numeric columns statistics:
+-------+---------+--------------------+------------------+------------------+--------------------+
|summary|   nconst|         primaryName|         birthYear|         deathYear|      knownForTitles|
+-------+---------+--------------------+------------------+------------------+--------------------+
|  count| 13040654|            13040654|          13040654|          13040654|            10442839|
|   mean|     NULL|                 NaN|1952.9593527326338| 1992.678400769587|                NULL|
| stddev|     NULL|                 NaN| 34.93445420136517|34.538771630283385|                NULL|
|   

In [12]:
# Title_Ratings:
print("Schema for Title_Ratings:")
title_ratings_df.printSchema()

print("\nColumns:")
print(title_ratings_df.columns)

num_columns = len(title_ratings_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = title_ratings_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
title_ratings_df.describe().show()

Schema for Title_Ratings:
root
 |-- tconst: string (nullable = true)
 |-- averageRating: float (nullable = true)
 |-- numVotes: integer (nullable = true)


Columns:
['tconst', 'averageRating', 'numVotes']

Number of columns: 3

Number of rows: 1373171

Numeric columns statistics:
+-------+---------+------------------+------------------+
|summary|   tconst|     averageRating|          numVotes|
+-------+---------+------------------+------------------+
|  count|  1373171|           1373171|           1373171|
|   mean|     NULL|  6.95877134629873|1038.0719903056502|
| stddev|     NULL|1.3831073386952943|17592.476561517775|
|    min|tt0000001|               1.0|                 5|
|    max|tt9916880|              10.0|           2822434|
+-------+---------+------------------+------------------+


In [13]:
# Title_Principals:
print("Schema for Title_Principals:")
title_principals_df.printSchema()

print("\nColumns:")
print(title_principals_df.columns)

num_columns = len(title_principals_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = title_principals_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
title_principals_df.describe().show()

Schema for Title_Principals:
root
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)


Columns:
['tconst', 'ordering', 'nconst', 'category']

Number of columns: 4

Number of rows: 59261891

Numeric columns statistics:
+-------+---------+-----------------+---------+--------+
|summary|   tconst|         ordering|   nconst|category|
+-------+---------+-----------------+---------+--------+
|  count| 59261891|         59261891| 59261891|59261891|
|   mean|     NULL|4.613541845973157|     NULL|    NULL|
| stddev|     NULL|2.787231004650719|     NULL|    NULL|
|    min|tt0000001|                1|nm0000001|   actor|
|    max|tt9916880|               10|nm9993718|  writer|
+-------+---------+-----------------+---------+--------+


In [14]:
# Title_Episode:
###print("Schema for Title_Episode:")
###title_episode_df.printSchema()

###print("\nColumns:")
###print(title_episode_df.columns)

###num_columns = len(title_episode_df.columns)
###print(f"\nNumber of columns: {num_columns}")

###num_rows = title_episode_df.count()
###print(f"\nNumber of rows: {num_rows}")

###print("\nNumeric columns statistics:")
###title_episode_df.describe().show()

In [15]:
#Title_Crew:
print("Schema for Title_Crew:")
title_crew_df.printSchema()

print("\nColumns:")
print(title_crew_df.columns)

num_columns = len(title_crew_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = title_crew_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
title_crew_df.describe().show()

Schema for Title_Crew:
root
 |-- tconst: string (nullable = true)
 |-- directors: string (nullable = true)


Columns:
['tconst', 'directors']

Number of columns: 2

Number of rows: 10343447

Numeric columns statistics:
+-------+---------+-------------------+
|summary|   tconst|          directors|
+-------+---------+-------------------+
|  count| 10343447|           10343447|
|   mean|     NULL|               NULL|
| stddev|     NULL|               NULL|
|    min|tt0000001|                 \N|
|    max|tt9916880|nm9993709,nm9993708|
+-------+---------+-------------------+


In [16]:
#Title_Basics:
print("Schema for Title_Basics:")
title_basics_df.printSchema()

print("\nColumns:")
print(title_basics_df.columns)

num_columns = len(title_basics_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = title_basics_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
title_basics_df.describe().show()

Schema for Title_Basics:
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: string (nullable = true)


Columns:
['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']

Number of columns: 9

Number of rows: 10345990

Numeric columns statistics:
+-------+---------+---------+--------------+-------------+-------------------+------------------+-----------------+-----------------+--------+
|summary|   tconst|titleType|  primaryTitle|originalTitle|            isAdult|         startYear|          endYear|   runtimeMinutes|  genres|
+-------+---------+---------+--------------+-------------+-------------------+------------------+-----

In [17]:
#Title_Akas:
print("Schema for Title_Akas:")
title_akas_df.printSchema()

print("\nColumns:")
print(title_akas_df.columns)

num_columns = len(title_akas_df.columns)
print(f"\nNumber of columns: {num_columns}")

num_rows = title_akas_df.count()
print(f"\nNumber of rows: {num_rows}")

print("\nNumeric columns statistics:")
title_akas_df.describe().show()

Schema for Title_Akas:
root
 |-- titleId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- types: string (nullable = true)


Columns:
['titleId', 'title', 'region', 'types']

Number of columns: 4

Number of rows: 37949764

Numeric columns statistics:
+-------+---------+-----------------+--------+--------+
|summary|  titleId|            title|  region|   types|
+-------+---------+-----------------+--------+--------+
|  count| 37949764|         37949764|37949764|37949761|
|   mean|     NULL|4.168411692889579|     NaN|    NULL|
| stddev|     NULL|3.996645990159475|     NaN|    NULL|
|    min|tt0000001|                1|   !Huff|      AD|
|    max|tt9916856|               99|🧠+🧘=❤️|      \N|
+-------+---------+-----------------+--------+--------+


In [18]:
# Бізнес питання: 
# 1 Який жанр фільмів є найпопулярніший (топ5)
# 2 Вивести топ 20 фільмів за оцінкою (мінімальна кількість оцінок 50000)
# 3 Які фільми найпопулярніші по кількості голосів (топ 20 ) та які їх оцінки?
# 4 популярніші короткі фільми чи довгі (менш як 110 хв - короткі) (за остання 50 років )
# 5 топ 10 фільмів кожного десятиліття, починаючи з 1970 року 
# 6 сума акторів котрі прожили більше 90 років та загальна сума акторів за весь час
# 7 вивести фільми які по тривалості потрапляють в діапазон: 2 -3 год, та які мають оцінку не більше 9.0
# 8 вивести фільми які по тривалості потрапляють в діапазон: 2 -3 год, та які мають суму голосів не менше 300000
# 9 вивести кількість фільмів які були зняті в 1895 році, кількість фільмів які були зняті в 2010, та скільки було знято в 2021

In [19]:
# 1) Який жанр фільмів є найпопулярніший (топ5)

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopGenres").getOrCreate()

# Розбивка жанрів, якщо декілька значень в колонці genres
title_basics = title_basics.withColumn("genres", explode(split(title_basics["genres"], ",")))

# Підрахунок кількості фільмів для кожного жанру
genre_counts = title_basics.groupBy("genres").count()

# Сортування за кількістю фільмів у зворотньому порядку
sorted_genres = genre_counts.sort(desc("count"))

# Виведення топ 5 жанрів
top_genres = sorted_genres.limit(5)
top_genres.show()

    # ПРИМІТКА:  тут показано кількість фільмів певного жанру не тільки по першому а й по другому і так далі пріоритету, тобто це може бути дія з елементами триллера чи драми, отже 1 фільм охоплює 3 категорії

+---------+-------+
|   genres|  count|
+---------+-------+
|    Drama|2939557|
|   Comedy|2062286|
|Talk-Show|1267772|
|    Short|1130666|
|  Romance| 977975|
+---------+-------+


In [27]:
# 2) Вивести топ 20 фільмів за оцінкою (мінімальна кількість оцінок 50000)

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopRatedMovies").getOrCreate()

# Завантаження даних з таблиці title_ratings
title_ratings = spark.read.option("delimiter", "\t").csv("./data/title_ratings.tsv", header=True, inferSchema=True)

# Завантаження даних з таблиці title_basics
title_basics = spark.read.option("delimiter", "\t").csv("./data/title_basics.tsv", header=True, inferSchema=True)

# Об'єднання таблиць title_ratings та title_basics за ключем tconst
merged_data = title_ratings.join(title_basics, title_ratings["tconst"] == title_basics["tconst"])

# Фільтрація за кількістю голосів (numVotes) більше або рівно 50000
filtered_data = merged_data.filter(merged_data["numVotes"] >= 50000)

# Сортування за оцінкою у зворотньому порядку за колонкою originalTitle
top_rated_movies = filtered_data.sort(desc("averageRating")).select("originalTitle", "averageRating")

# Обмеження результату до топ-20 фільмів
top_rated_movies = top_rated_movies.limit(20)

# Запис топ-фільмів у CSV файл
#top_rated_movies.write.csv("D:/BDProject/data/top_rated_movies2", header=True)

# Виведення топ фільмів
top_rated_movies.show(truncate=False)


# вибрав спеціально мінімум 50т голосів, шоб не було такого що видало фільм з оцінкою 10 і вона там одна єдина, вивело досить непогані фільми в яких найвищий рейтинг в лізі 'малої розпізнаваності' 

+------------------------+-------------+
|originalTitle           |averageRating|
+------------------------+-------------+
|Ozymandias              |10.0         |
|Plan and Execution      |9.9          |
|Face Off                |9.9          |
|Battle of the Bastards  |9.9          |
|Felina                  |9.9          |
|The Winds of Winter     |9.9          |
|The Rains of Castamere  |9.9          |
|Saul Gone               |9.8          |
|Midnight Sun            |9.8          |
|Chapter 16: The Rescue  |9.8          |
|Memories of the Future  |9.8          |
|Hardhome                |9.8          |
|The War Hammer Titan    |9.8          |
|Perfect Game            |9.8          |
|Declaration of War      |9.8          |
|To'hajiilee             |9.8          |
|Hero                    |9.8          |
|Vichnaya Pamyat         |9.8          |
|That Day                |9.7          |
|From You, 2000 Years Ago|9.7          |
+------------------------+-------------+


In [21]:
# 3) Які фільми найпопулярніші по кількості голосів (топ 20 ) та які їх оцінки?

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopRatedMovies").getOrCreate()

# Завантаження даних з таблиць title_ratings та title_basics
title_ratings = spark.read.option("delimiter", "\t").csv("./data/title_ratings.tsv", header=True, inferSchema=True)
title_basics = spark.read.option("delimiter", "\t").csv("./data/title_basics.tsv", header=True, inferSchema=True)

# Об'єднання таблиць title_ratings та title_basics за ключем tconst
merged_data = title_ratings.join(title_basics, title_ratings["tconst"] == title_basics["tconst"])

# Сортування за кількістю голосів (numVotes) у зворотньому порядку
most_rated_movies = merged_data.sort(desc("numVotes")).select("originalTitle", "numVotes", "averageRating")

# Запис топ-20 найпопулярніших фільмів в CSV файл
#most_rated_movies.limit(20).write.csv("D:/BDProject/data/top_rated_movies3", header=True)
most_rated_movies.show(truncate=False)

# бачимо що у високо обговорюваних фільмах також високий рейтинг, але не такий великий як міг бути 

+-------------------------------------------------+--------+-------------+
|originalTitle                                    |numVotes|averageRating|
+-------------------------------------------------+--------+-------------+
|The Shawshank Redemption                         |2822434 |9.3          |
|The Dark Knight                                  |2803998 |9.0          |
|Inception                                        |2488802 |8.8          |
|Fight Club                                       |2257826 |8.8          |
|Game of Thrones                                  |2224985 |9.2          |
|Forrest Gump                                     |2198531 |8.8          |
|Pulp Fiction                                     |2164328 |8.9          |
|Breaking Bad                                     |2065145 |9.5          |
|Interstellar                                     |2015591 |8.7          |
|The Matrix                                       |2006463 |8.7          |
|The Godfather           

In [35]:
# 4) Популярніші короткі фільми чи довгі (менше 110  хв - короткі) (за остання 50 років) (за кількістю голосів)

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("FilmVotes").getOrCreate()

# Визначення поточного року
current_year = 2023

# Фільтрація за умовами (останні 50 років)
recent_movies = title_basics.filter((title_basics["startYear"] >= current_year - 50) & (title_basics["startYear"] <= current_year))

# Розділення на короткі та довгі фільми
short_movies = recent_movies.filter(recent_movies["runtimeMinutes"] < 110)
long_movies = recent_movies.filter(recent_movies["runtimeMinutes"] >= 110)

# Загальна кількість голосів для коротких та довгих фільмів
total_votes_short = (
    short_movies.join(title_ratings, short_movies["tconst_basics"] == title_ratings["tconst"])
    .agg(sum("numVotes").alias("total_votes_short"))
    .first()
    .total_votes_short
)

total_votes_long = (
    long_movies.join(title_ratings, long_movies["tconst_basics"] == title_ratings["tconst"])
    .agg(sum("numVotes").alias("total_votes_long"))
    .first()
    .total_votes_long
)

print(f"Total votes under short films (runtime < 110 min): {total_votes_short}")
print(f"Total votes under long films (runtime >= 110 min): {total_votes_long}")

# Запис результатів в CSV файл
result_df_votes = spark.createDataFrame([(total_votes_short, total_votes_long)], ["total_votes_short", "total_votes_long"])
#result_df_votes.write.csv("D:/BDProject/data/film_votes_results", header=True)


# бачимо що попри позибку в голосах за кородкі та довгі фільми, не можна сказати на певне що коротку фільми популярніші, так як логічно що на корродші фільми сумарно йде менше часу отже і більше голосів 

Total votes under short films (runtime < 110 min): 723210164
Total votes under long films (runtime >= 110 min): 599000327


In [26]:
# 5) топ 10 фільмів кожного десятиліття, починаючи з 1970 року
# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopMoviesByDecade").getOrCreate()

# Додаємо псевдоніми до полів для уникнення конфлікту
title_ratings = title_ratings.withColumnRenamed("tconst", "rating_tconst")
title_basics = title_basics.withColumnRenamed("tconst", "basics_tconst")

# Об'єднуємо таблиці за tconst
joined_data = title_ratings.join(title_basics, title_ratings.rating_tconst == title_basics.basics_tconst)

# Фільтрація за роками та розбиття на десятиліття
filtered_data = joined_data.filter((col("startYear") >= 1970) & (col("startYear") < 2023))
filtered_data = filtered_data.withColumn("decade", ((col("startYear") - 1970) / 10).cast("int") * 10 + 1970)

# Використовуємо вікно для нумерації фільмів у кожному десятилітті за кількістю голосів
window_spec = Window.partitionBy("decade").orderBy(desc("numVotes"))
ranked_data = filtered_data.withColumn("rank", row_number().over(window_spec))

# Вибірка топ-фільмів для кожного десятиліття
top_movies_by_decade = ranked_data.filter(col("rank") <= 10)

# Виведення результатів
top_movies_by_decade.select("decade", "primaryTitle", "numVotes").orderBy("decade", "rank").show(n=60, truncate=False)

# Запис топ-фільмів у CSV файл
#top_movies_by_decade.select("decade", "primaryTitle", "numVotes").orderBy("decade", "rank").write.csv("D:/BDProject/data/top_movies_by_decade5", header=True)


+------+-------------------------------------------------+--------+
|decade|primaryTitle                                     |numVotes|
+------+-------------------------------------------------+--------+
|1970  |The Godfather                                    |1967260 |
|1970  |Star Wars: Episode IV - A New Hope               |1423085 |
|1970  |The Godfather Part II                            |1335344 |
|1970  |One Flew Over the Cuckoo's Nest                  |1050805 |
|1970  |Alien                                            |928051  |
|1970  |Taxi Driver                                      |895306  |
|1970  |A Clockwork Orange                               |865639  |
|1970  |Apocalypse Now                                   |696759  |
|1970  |Jaws                                             |645700  |
|1970  |Rocky                                            |615074  |
|1980  |Star Wars: Episode V - The Empire Strikes Back   |1352012 |
|1980  |Back to the Future                      

In [16]:
# 6) сума акторів котрі прожили більше 90 років та загальна сума акторів за весь час

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("ActorAge").getOrCreate()

# Завантаження даних з таблиці name_basics
name_basics = spark.read.option("delimiter", "\t").csv("./data/name_basics.tsv", header=True, inferSchema=True)

# Розрахунок тривалості життя (різниця між роком смерті та роком народження)
name_basics = name_basics.withColumn("age", when(col("deathYear") != "\\N", col("deathYear").cast("int") - col("birthYear").cast("int")).otherwise(None).cast(IntegerType()))

# Розрахунок кількості акторів, які прожили більше 90 років
actors_over_90 = name_basics.filter(col("age") > 90)
count_actors_over_90 = actors_over_90.count()

# Розрахунок кількості всіх акторів
count_all_actors = name_basics.count()

# Запис результатів в CSV файл
result_df = spark.createDataFrame([(count_actors_over_90, count_all_actors)], ["count_actors_over_90", "count_all_actors"])
#result_df.write.csv("D:/BDProject/data/actor_age_results6", header=True)

result_df.show(truncate=False)
# навіть попри те що вибірка акторів у нас 13 млн за всі роки, тільки 20000 акторів з них можна назвати довгожителями 
 

+--------------------+----------------+
|count_actors_over_90|count_all_actors|
+--------------------+----------------+
|19964               |13040654        |
+--------------------+----------------+


In [32]:
# 7) вивести фільми які по тривалості потрапляють в діапазон: 2 -3 год, та які мають оцінку не більше 9.0
# Приєднання таблиць за допомогою tconst

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopMovies").getOrCreate()

# Завантаження даних з таблиць title_ratings та title_basics
title_ratings = spark.read.option("delimiter", "\t").csv("./data/title_ratings.tsv", header=True, inferSchema=True)
title_basics = spark.read.option("delimiter", "\t").csv("./data/title_basics.tsv", header=True, inferSchema=True)

# Перейменуємо стовпець у title_basics для уникнення конфлікту імен
title_basics = title_basics.withColumnRenamed("tconst", "tconst_basics")

# Приєднання таблиць за допомогою tconst
joined_df = title_ratings.join(title_basics, title_ratings["tconst"] == title_basics["tconst_basics"])

# Фільтрація фільмів за заданими умовами
filtered_movies = joined_df.filter((col("runtimeMinutes") > 120) & (col("runtimeMinutes") < 180) & (col("averageRating") > 9.0))

# Отримання кількості фільмів та їхніх назв
number_of_movies = filtered_movies.count()
movie_titles = filtered_movies.select("primaryTitle")

# Виведення результату
print(f"Кількість фільмів: {number_of_movies}")
print("Назви фільмів:")
movie_titles.show(truncate=False)

# Запис результату у CSV файл
#movie_titles.limit(20).coalesce(1).write.csv("D:/BDProject/data/top_movies_output7", header=True)



# бачимо що критерій (якість оцінки) нам не допоміг знайти та видати хороші фільми та серіали, видав якісь 3 сортні результати(індуські фільми та серіали), висновок: індуси псують статистику)  

Кількість фільмів: 656
Назви фільмів:
+----------------------------------------------+
|primaryTitle                                  |
+----------------------------------------------+
|Big Booty White Girls                         |
|Der doppelte Moritz                           |
|David Copperfield                             |
|Eine große Familie                            |
|Der Geisterbräu                               |
|Ein Wintermärchen                             |
|Pink Floyd: P. U. L. S. E. Live at Earls Court|
|Mexikanische Revolution                       |
|Persée                                        |
|Edadugula Bandham                             |
|Al Capone im deutschen Wald                   |
|Desh Devi                                     |
|Turandot                                      |
|Das Herrschaftskind                           |
|Veera Kesari                                  |
|Pelléas et Mélisande                          |
|New York City Opera: The Merry

In [33]:
# 8) вивести фільми які по тривалості потрапляють в діапазон: 2 -3 год, та які мають суму голосів не менше 300000
# Приєднання таблиць за допомогою tconst
# Приєднання таблиць за допомогою tconst

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("TopMovies").getOrCreate()

# Завантаження даних з таблиць title_ratings та title_basics
title_ratings = spark.read.option("delimiter", "\t").csv("./data/title_ratings.tsv", header=True, inferSchema=True)
title_basics = spark.read.option("delimiter", "\t").csv("./data/title_basics.tsv", header=True, inferSchema=True)

# Перейменуємо стовпець у title_basics для уникнення конфлікту імен
title_basics = title_basics.withColumnRenamed("tconst", "tconst_basics")

# Приєднання таблиць за допомогою tconst
joined_df = title_ratings.join(title_basics, title_ratings["tconst"] == title_basics["tconst_basics"])

# Фільтрація фільмів за заданими умовами
filtered_movies = joined_df.filter(
    (col("runtimeMinutes") >= 120) & (col("runtimeMinutes") <= 180) & (col("numVotes") >= 300000)
)

# Отримання кількості фільмів та їхніх назв
number_of_movies = filtered_movies.count()
movie_titles = filtered_movies.select("primaryTitle").limit(20)

# Виведення результату
print(f"Кількість фільмів: {number_of_movies}")
print("Назви фільмів:")
movie_titles.show(truncate=False)

# Запис результату у CSV файл
#movie_titles.coalesce(1).write.csv("D:/BDProject/data/top_movies_output8", header=True)



# як ми бачимо порівняно з критерієм(оцінкою) критерій (голоси) працює краще для знаходження крутих фільмів, в аутпуті видає справді круті оскароносні фільми, з чого можна зробити висновок що не завжди якщо навпроти фільму стоїть висока оцінка це значить що фільм чи серіал вартий уваги, бо бачимо що нікудишній індуський серіал отримує оцінку 9.0 і вище, хоча хороший фільм який є просто більш обговорюваним в суспільстві(кількість голосів під фільмом) але має трохи нижчу оцінку, наприлад(8.2-8.7) - є очевидно кращий, тому критерій кількість голосів також кращий для означення хорошого чи поганого фільму

Кількість фільмів: 350
Назви фільмів:
+--------------------------------------------+
|primaryTitle                                |
+--------------------------------------------+
|Star Wars: Episode III - Revenge of the Sith|
|Saving Private Ryan                         |
|Sin City                                    |
|The Devil's Advocate                        |
|Cast Away                                   |
|Zodiac                                      |
|Scarface                                    |
|Casino                                      |
|Brokeback Mountain                          |
|Good Will Hunting                           |
|Star Wars: Episode VI - Return of the Jedi  |
|Love Actually                               |
|The Holiday                                 |
|Apollo 13                                   |
|Harry Potter and the Half-Blood Prince      |
|Goodfellas                                  |
|American Beauty                             |
|The Last Samurai     

In [25]:
# 9) вивести кількість фільмів які були зняті в 1895 році, кількість фільмів які були зняті в 2010, та скільки було знято в 2021

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("FilmCount").getOrCreate()

# Завантаження даних з таблиці title_basics
title_basics = spark.read.option("delimiter", "\t").csv("./data/title_basics.tsv", header=True, inferSchema=True)

# Фільтрація фільмів за роками виробництва та підготовка даних для об'єднання
films_data = [
    Row(year=1895, count=title_basics.filter(col("startYear") == 1895).count()),
    Row(year=2010, count=title_basics.filter(col("startYear") == 2010).count()),
    Row(year=2021, count=title_basics.filter(col("startYear") == 2021).count())
]

# Створення DataFrame з підготовлених даних
films_df = spark.createDataFrame(films_data)

# Запис у CSV файл (все в один файл)
#films_df.coalesce(1).write.csv("./data/films_count9", header=True)

# Виведення результату
films_df.show(truncate=False)




# навіть попри те що фільми в кінці 19 століння тривали приблизно кілька хвилин а не як зараз по кілька годин, все ж сума фільмів того чавсу набагато менша ніж є зараз, також можна побачити як розвинувся кінематограф порівняно з попереднім десятиліттям і зараз, майже в 2 рази більше знятих фільмів станом на 2021 рік порівняно з 2010 роком

+----+------+
|year|count |
+----+------+
|1895|114   |
|2010|253946|
|2021|472724|
+----+------+


In [ ]:
#Load all results of transformations to .csv files
# Який жанр фільмів є найпопулярніший (топ5)?
top_genres.write.csv("D:/BDProject/data/csv", header= True)

# Bивести топ 20 фільмів за оцінкою (мінімальна кількість оцінок 50000)
top_rated_movies.write.csv("D:/BDProject/data/top_rated_movies2", header=True)

# Які найпопулярніші фільми, по кількості голосів (топ 20) та які їх оцінки?
most_rated_movies.limit(20).write.csv("D:/BDProject/data/top_rated_movies3", header=True)

# За останні 50 років популярніші короткі(менше 110хв) чи довгі(більше 110хв) фільми (за кількістю голосів)?
result_df_votes.write.csv("D:/BDProject/data/film_votes_results", header=True)

# Які фільми були найпопулярнішими(топ10) кожного десятиліття (починаючи з 1970 року)?
top_movies_by_decade.select("decade", "primaryTitle", "numVotes").orderBy("decade", "rank").write.csv("D:/BDProject/data/top_movies_by_decade5", header=True)

# Яка сума акторів котрі прожили більше 90 років та яка загальна сума акторів за весь час?
result_df.write.csv("D:/BDProject/data/actor_age_results6", header=True)

# Які фільми по тривалості потрапляють в діапазон 2-3 год та мають оцінку не більше 9.0?
movie_titles.limit(20).coalesce(1).write.csv("D:/BDProject/data/top_movies_output7", header=True)

# Які фільми по тривалості потрапляють в діапазон 2-3 год та мають суму голосів не менше 300000?
movie_titles.coalesce(1).write.csv("D:/BDProject/data/top_movies_output8", header=True)

# Скільки фільмів було знято в 1895, 2010 та в 2021 роках відповідно?

films_df.coalesce(1).write.csv("./data/films_count9", header=True)
